In [1]:
import time
import pyaubo_sdk
class Aubo:
    def __init__(self):
        self.robot = None
        self.cal_home_pose = None
        self.user_coord = None
        self.tcp_offset = None
        self.cal_point3 = None
        self.cal_point2 = None
        self.cal_point1 = None
        self.ee_line_velc = None
        self.ee_line_acc = None
        self.joint_velc = None
        self.joint_acc = None
        self.robot_ip = None
        self.tcp_rpy = None
        self.rpc_client,self.script_client = self._robot_init()

    def _robot_init(self):
        rpc_client = pyaubo_sdk.RpcClient()
        rpc_client.setRequestTimeout(300000)
        script_client = pyaubo_sdk.ScriptClient()
        return rpc_client, script_client


    def login(self,robot_ip):
        rpc_port = 30004
        script_port = 30002
        result_rpc = self.rpc_client.connect(robot_ip, rpc_port)
        result_script = self.script_client.connect(robot_ip,script_port)
        if result_rpc != 0 and result_script !=0:
            return False
        else:
            self.rpc_client.login("aubo", "123456")
            self.script_client.login("aubo", "123456")
            if self.rpc_client.hasLogined() and self.script_client.hasLogined():
                robot_name = self.rpc_client.getRobotNames()[0]
                self.robot = self.rpc_client.getRobotInterface(robot_name)


        return result_rpc == 0 and result_script ==0

    # 阻塞
    def waitArrival(self):
        cnt = 0
        while self.robot.getMotionControl().getExecId() == -1:
            cnt += 1
            if cnt > 5:
                print("Motion fail!")
                return -1
            time.sleep(0.05)
            print("getExecId: ", self.robot.getMotionControl().getExecId())
        id = self.robot.getMotionControl().getExecId()
        while True:
            id1 = self.robot.getMotionControl().getExecId()
            if id != id1:
                break
            time.sleep(0.05)



    def goHome(self):
        ret = self.robot.getMotionControl().moveJoint(self.cal_home_pose, self.joint_acc, self.joint_velc, 0, 0)
        return ret

    def getStatus(self):
        return self.robot.getRobotState().isSteady()

    def stop(self):
        print('robot stop')
        # return self.robot.getMotionControl().stopLine(self.ee_line_acc,self.ee_line_velc)
        return self.robot.getMotionControl().stopJoint(self.joint_acc)

    def robot_startup(self):
        if not self.robot.getRobotState().isPowerOn():
            if 0 == self.robot.getRobotManage().poweron():
                print("The robot is requesting power-on!")
                if 0 == self.robot.getRobotManage().startup():  # 接口调用: 发起机器人启动请求
                    print("The robot is requesting startup!")
                    # 循环直至机械臂松刹车成功
                    while 1:
                        robot_mode = self.robot.getRobotState().getRobotModeType()  # 接口调用: 获取机器人的模式类型
                        print("Robot current mode: %s" % (robot_mode.name))
                        if robot_mode == pyaubo_sdk.RobotModeType.Running:
                            break
                        time.sleep(1)
    def robot_poweroff(self):
        if 0 == self.robot.getRobotManage().poweroff():
            print("The robot is requesting power-off!")
    def exit(self):
        if self.rpc_client.hasConnected():
            self.rpc_client.disconnect()
        if self.script_client.hasConnected():
            self.script_client.disconnect()
        return 0



In [3]:
aubo = Aubo()
aubo.login(robot_ip="192.168.0.106")
if aubo.rpc_client.hasLogined():
    aubo.robot_startup()


[Info] [connector_asio.h:110] Connect to 192.168.0.106:30004
[Info] [rpc.cpp:116] SDK INTERFACE VERSION: 0.23.0
[Info] [rpc.cpp:117] SERVER INTERFACE VERSION: 0.23.1
[Info] [connector_asio.h:154] RPC Client login
[Info] [connector_script_tcp.h:224] Script Client login
[Info] [connector_script_tcp.h:53] Start aubo_script recv polling...


In [6]:
#打印当前关节角
current_positions = aubo.robot.getRobotState().getJointPositions()
print(current_positions)

[-1.3762624598241155, 0.37081929445481526, 1.9833614483449218, 1.6306988926759154, 1.5603791271682912, 0.03204389293592429]


In [ ]:
aubo.exit()

In [23]:
from scipy.spatial.transform import Rotation as R
import numpy as np
# 设置 numpy 的打印选项
np.set_printoptions(suppress=True)
rot = R.from_euler('xyz', [3.11258322,  0.029991,   -1.57161497], degrees=False)
rot_quat = rot.as_quat()
print(rot_quat)

[ 0.70681733 -0.70708849 -0.02085663 -0.00035557]


In [13]:
quat = [-0.7067991252811548, 0.7071066914673226, 0.02085656975139953, 0.0003562051881106084]
rot = R.from_quat(quat).as_euler('xyz', degrees=False)
print(rot)

[ 3.11258343  0.02999106 -1.57166646]


In [21]:
rot = R.from_euler('ZYX', [ -1.57161497, 0.029991,   3.11258322], degrees=False)
rot_quat = rot.as_quat()
print(rot_quat)

[ 0.70681733 -0.70708849 -0.02085663 -0.00035557]


In [42]:
quat = [-0.7067991252811548, 0.7071066914673226, 0.02085656975139953, 0.0003562051881106084]
rot = R.from_quat(quat).as_euler('xyz', degrees=False)
print(rot)

[ 3.11258343  0.02999106 -1.57166646]


In [34]:
quat = [-0.02085663, -0.70708849, 0.70681733, -0.00035557]
rot = R.from_quat(quat).as_euler('xyz', degrees=False)
print(rot)

[-1.57161496  0.02999099  3.11258322]


In [39]:
from tf_transformations import euler_from_quaternion,quaternion_from_euler

quaternion = [-0.7067991252811548, 0.7071066914673226, 0.02085656975139953, 0.0003562051881106084]

euler = euler_from_quaternion(quaternion, axes='sxyz')
print(euler)
quat = quaternion_from_euler(*euler)
print(quat)
euler2 = euler_from_quaternion(quat, axes='sxyz')
print(euler2)

(3.112583432455337, 0.029991056431805937, -1.571666457439996)
(0.7067991252811547, -0.7071066914673226, -0.020856569751399538, -0.0003562051881106001)
(3.112583432455337, 0.029991056431805933, -1.571666457439996)


In [51]:
rot = R.from_euler('xyz', [ 3.11258322, 0.029991, -1.57161497  ], degrees=False)
rot_quat = rot.as_quat()
print(rot_quat)

[ 0.70681733 -0.70708849 -0.02085663 -0.00035557]


In [54]:
quat = quaternion_from_euler(3.11258322, 0.029991, -1.57161497,axes='sxyz')
print(quat)

(0.706817328650944, -0.7070884939671822, -0.020856634092090195, -0.00035557321892849304)


In [45]:
# 欧拉角
euler_xyz = [3.11258343, 0.02999106, -1.57166646]
euler_zyx = [-1.57166646, 0.02999106, 3.11258343]
euler_xyz_intrinsic = [3.11258343, 0.02999106, -1.57166646]
euler_zyx_intrinsic = [-1.57166646, 0.02999106, 3.11258343]
# 转换为四元数（静态外旋）
quat_xyz_extrinsic = R.from_euler('XYZ', euler_xyz, degrees=False).as_quat()
quat_zyx_extrinsic = R.from_euler('ZYX', euler_zyx, degrees=False).as_quat()
# 转换为四元数（动态内旋）
quat_xyz_intrinsic = R.from_euler('xyz', euler_xyz_intrinsic, degrees=False).as_quat()

quat_zyx_intrinsic = R.from_euler('zyx', euler_zyx_intrinsic, degrees=False).as_quat()
print("Quat from Extrinsic XYZ:", quat_xyz_extrinsic)
print("Quat from Extrinsic ZYX:", quat_zyx_extrinsic)
print("Quat from Intrinsic xyz:", quat_xyz_intrinsic)
print("Quat from Intrinsic zyx:", quat_zyx_intrinsic)

Quat from Extrinsic XYZ: [0.70649142 0.70741413 0.00033806 0.02085687]
Quat from Extrinsic ZYX: [ 0.70679912 -0.70710669 -0.02085657 -0.00035621]
Quat from Intrinsic xyz: [ 0.70679912 -0.70710669 -0.02085657 -0.00035621]
Quat from Intrinsic zyx: [0.70649142 0.70741413 0.00033806 0.02085687]


In [55]:
import numpy as np
from tf_transformations import quaternion_from_euler
from scipy.spatial.transform import Rotation as R

# 欧拉角 [roll, pitch, yaw]
angles = [3.11258343, 0.02999106, 1.57166646]

# 使用 tf2 库
# 默认使用静态（外旋）顺序 'sxyz'
quat_tf2_sxyz = quaternion_from_euler(angles[0], angles[1], angles[2], 'sxyz')
quat_tf2_rzyx = quaternion_from_euler(angles[0], angles[1], angles[2], 'rzyx')

# 使用 scipy 库
# 使用内旋顺序 'xyz'
quat_scipy_xyz = R.from_euler('xyz', angles, degrees=False).as_quat()

# 使用外旋顺序 'zyx'
quat_scipy_zyx = R.from_euler('zyx', angles, degrees=False).as_quat()

print("Quaternion from tf2 (static, sxyz):", quat_tf2_sxyz)
print("Quaternion from tf2 (dynamic, rzyx):", quat_tf2_rzyx)
print("Quaternion from scipy (inner, xyz):", quat_scipy_xyz)
print("Quaternion from scipy (outer, zyx):", quat_scipy_zyx)

# 检查四元数是否等价
def check_quaternion_equivalence(quat1, quat2):
    return np.allclose(quat1, quat2) or np.allclose(-quat1, quat2)

print("Are tf2 (sxyz) and scipy (xyz) equivalent?", check_quaternion_equivalence(quat_tf2_sxyz, quat_scipy_xyz))
print("Are tf2 (rzyx) and scipy (zyx) equivalent?", check_quaternion_equivalence(quat_tf2_rzyx, quat_scipy_zyx))

Quaternion from tf2 (static, sxyz): (0.7064914153128985, 0.7074141337407512, -0.0003380574808932082, 0.020856873930188473)
Quaternion from tf2 (dynamic, rzyx): (-0.0003380574808932082, 0.7074141337407512, 0.7064914153128985, 0.020856873930188473)
Quaternion from scipy (inner, xyz): [ 0.70649142  0.70741413 -0.00033806  0.02085687]
Quaternion from scipy (outer, zyx): [ 0.02085657 -0.70710669  0.70679912 -0.00035621]
Are tf2 (sxyz) and scipy (xyz) equivalent? True


TypeError: bad operand type for unary -: 'tuple'

In [1]:
def process_objects(**kwargs):
    # 获取 kwargs 中的元素
    object_1 = kwargs.get('object_1', '未指定')
    object_2 = kwargs.get('object_2', '未指定')

    # 根据元素执行不同的操作
    if object_1 == '红色方块' and object_2 == '蓝色方块':
        print("找到了红色方块和蓝色方块")
    else:
        print("对象不符合预期")

    # 处理并打印
    print(f"对象1: {object_1}")
    print(f"对象2: {object_2}")

# 调用函数并传递 kwargs
kwargs = {'object_1': '红色方块', 'object_2': '蓝色方块'}
process_objects(**kwargs)

找到了红色方块和蓝色方块
对象1: 红色方块
对象2: 蓝色方块


In [2]:

import json
import ast

# 原始的 Python 字典字符串
json_string = "{'function_call': [{'name': 'vlm_pick_and_place', 'params': '{\"object_1\": \"红色方块\", \"object_2\": \"蓝色方块\"}'}]}"

# 将 Python 字典字符串转换为字典
data_dict = ast.literal_eval(json_string)

# 将字典转换为 JSON 字符串
json_data = json.dumps(data_dict)

# 解析 JSON 字符串
parsed_data = json.loads(json_data)

# 打印解析后的数据
print(parsed_data)

# 提取具体的参数
function_call = parsed_data.get('function_call', [])
for func in function_call:
    name = func.get('name', '')
    params_str = func.get('params', '{}')
    params = json.loads(params_str)
    print(f"Function Name: {name}")
    print(f"Params: {params}")

{'function_call': [{'name': 'vlm_pick_and_place', 'params': '{"object_1": "红色方块", "object_2": "蓝色方块"}'}]}
Function Name: vlm_pick_and_place
Params: {'object_1': '红色方块', 'object_2': '蓝色方块'}
